-------------------------------------------------------------------------------

- **Nome:** Renato Andrade Mosqueira Furtado
- **e-mail:** renato.andrade@engenharia.ufjf.br
- **Proposta:** Alocação ótima de filtros passivos série para minimização da distorção harmônica total (THD) via técnicas inteligentes.

-------------------------------------------------------------------------------

# **Informações importantes**

**O presente trabalho está dividido da seguinte forma:**

- **Bibliotecas:** seção na qual é feita a importação das bibliotecas que serão úteis ao longo do código. São elas:
 - Numpy: biblioteca que trabalha com cálculos matriciais;
 - Pandas: biblioteca que trabalha com base de dados e criação de tabelas;
 - Matplotlib: biblioteca que auxilia na criação de gráficos;
 - Cmath: biblioteca que possui operadores matemáticos diversos.

- **Fluxo de potência:** seção na qual é feita toda formulação e aplicação do fluxo de potência na frequência fundamental. Nessa parte, há a entrada de dados do sistema em uso, cálculo da matriz $Y_{barra}$ e, por fim, uma função genérica que calcula o fluxo de potência, entregando como resultado o estado operativo da rede;

- **Análise harmônica em sistemas de potência:** seção na qual é feita toda formulação e cálculos essenciais para aplicação do fluxo de potência harmônico. Nessa parte, há os dados de entrada da fonte harmônica, cálculo das matrizes $Y$ dos elementos *shunts* e nodais para cada ordem harmônica e uma função genérica para cálculo do fluxo de potência harmônica (Método da Compensação das Correntes);

- **Algoritmo de Otimização Aritmética - Alocação de Filtro:** seção na qual é feita o processo de alocação de filtro e a escolha dos parâmetros R, L e C via Algoritmo de Otimização Aritmética. Nessa parte, há a definição dos limites mínimos e máximos das variáveis a serem otimizadas, dos parâmetros da meta-heurística e, por fim, o processo iterativo é ilustrado. Nessa parte, o usuário pode escolher em qual tipo de função objetivo irá trabalhar, ou seja, se utilizará o caso 1 (sem penalização) ou caso 2 (com penalização);

- **Gráficos:** seção na qual são explicitados os gráficos da convergência da otimização e também a comparação entre os *THDs* antes e depois. Na parte "Comparação THD por barra e modelagem da Função Objetivo", para executar corretamente, deve-se aplicar tanto o caso 1 e caso 2 antes da execução.

# **Bibliotecas**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cmath import*

# **Fluxo de potência**

## Entrada de dados

### Dados de barra

In [ ]:
#              N° Barra - Tipo - V (pu) - Ângulo (°) - Pk gerada (pu) - Qk gerada (pu) - Pk demandada (pu) - Qk demandada (pu)

numero_barra = [1,2,3,4,5,6,7,8,9,10,11,12,13]

tipo_barra = ['Slack','PV','PQ','PQ','PQ','PQ','PQ','PQ','PQ','PQ','PQ','PQ','PQ']

tensao_barra = [1.000,0.995,'x','x','x','x','x','x','x','x','x','x','x']

angulo_barra = [0.00,'x','x','x','x','x','x','x','x','x','x','x','x']

p_gerada_barra = ['x',0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]

q_gerada_barra = ['x','x',0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]

p_demandada_barra = ['x',0.00,0.00,0.06,0.2240,0.00,0.1150,0.1310,0.00,0.0810,0.00,0.0370,0.28]

q_demandada_barra = ['x','x',0.00,0.053,0.20,0.00,0.0290,0.1130,0.00,0.080,0.00,0.033,0.25]

shunt_barra = [0,0,0,0,1j*0.6,0.0,0,0,0,0,0,0,0]

Dados_barra = pd.DataFrame(data = np.transpose([numero_barra,tipo_barra,tensao_barra,angulo_barra,p_gerada_barra,q_gerada_barra,p_demandada_barra,q_demandada_barra,shunt_barra]),columns=['N° barra','Tipo','V (pu)','Ângulo (°)','Pg (pu)','Qg (pu)','Pd (pu)','Qd (pu)','Shunt (pu)'])

Dados_barra.set_index('N° barra',inplace=True)

Dados_barra.index = Dados_barra.index.astype('int')
Dados_barra['Shunt (pu)'] = Dados_barra['Shunt (pu)'].astype('complex')

Dados_barra

,Tipo,V (pu),Ângulo (°),Pg (pu),Qg (pu),Pd (pu),Qd (pu),Shunt (pu)
N° barra,,,,,,,,
1,Slack,1.0,0.0,x,x,x,x,0.0+0.0j
2,PV,0.995,x,0.2,x,0.0,x,0.0+0.0j
3,PQ,x,x,0.0,0.0,0.0,0.0,0.0+0.0j
4,PQ,x,x,0.0,0.0,0.06,0.053,0.0+0.0j
5,PQ,x,x,0.0,0.0,0.224,0.2,0.0+0.6j
6,PQ,x,x,0.0,0.0,0.0,0.0,0.0+0.0j
7,PQ,x,x,0.0,0.0,0.115,0.029,0.0+0.0j
8,PQ,x,x,0.0,0.0,0.131,0.113,0.0+0.0j
9,PQ,x,x,0.0,0.0,0.0,0.0,0.0+0.0j


### Dados de linhas

- **Tap:** referenciado sempre para a barra *para*;
- **Shunt de linha:** deve-se colocar o valor $B/2$.

In [ ]:
#              De - Para - R (pu) - X (pu) - B/2 (pu) - Tap - Defasagem (°)

DE = [1,5,5,5,5,3,2,6,6,9,11,11]

PARA = [3,2,6,9,11,5,4,7,8,10,12,13]

Z_base = 1

resistencia = [0.00139,0.00122,0.00075,0.00157,0.00109,0.003132,0.063953333,0.059184,0.043142028,0.058286666,0.055753333,0.012181333]
resistencia = [i/Z_base for i in resistencia]

reatancia = [0.00296,0.00243,0.00063,0.00131,0.00091,0.053241333,0.37796,0.355104,0.345142029,0.378873333,0.3624,0.14616]
reatancia = [i/Z_base for i in reatancia]

shunt = [0,0,0,0,0,0,0,0,0,0,0,0]

Tap = [1,1,1,1,1,1/1.0,1/0.974637681,1/0.974637681,1/0.95,1/0.974637681,1/1.0,1/0.95]

Fi = [0,0,0,0,0,0,0,0,0,0,0,0]

Dados_linha = pd.DataFrame(data = np.transpose([DE,PARA,resistencia,reatancia,shunt,Tap,Fi]),columns=['De','Para','R (pu)','X (pu)','B/2 (pu)','Tap','Defasagem (°)'])

Dados_linha

,De,Para,R (pu),X (pu),B/2 (pu),Tap,Defasagem (°)
0,1.0,3.0,0.001390,0.002960,0.0,1.000000,0.0
1,5.0,2.0,0.001220,0.002430,0.0,1.000000,0.0
2,5.0,6.0,0.000750,0.000630,0.0,1.000000,0.0
3,5.0,9.0,0.001570,0.001310,0.0,1.000000,0.0
4,5.0,11.0,0.001090,0.000910,0.0,1.000000,0.0
5,3.0,5.0,0.003132,0.053241,0.0,1.000000,0.0
6,2.0,4.0,0.063953,0.377960,0.0,1.026022,0.0
7,6.0,7.0,0.059184,0.355104,0.0,1.026022,0.0
8,6.0,8.0,0.043142,0.345142,0.0,1.052632,0.0
9,9.0,10.0,0.058287,0.378873,0.0,1.026022,0.0


### Cálculo da Matriz Ybarra

In [ ]:
def calculo_ybus(Dados_barra,Dados_linha):

  Dados_barra = Dados_barra.copy()
  Dados_linha = Dados_linha.copy()

  Ybarra = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)),dtype='complex_')

  for k in Dados_barra.index:
    for m in Dados_barra.index:
      if k==m:
        for p in Dados_linha.index:
          if Dados_linha['De'][p]==k or Dados_linha['Para'][p]==k:
            if Dados_linha['De'][p]==k:
              Ybarra[k-1][m-1] = Ybarra[k-1][m-1] + (Dados_linha['Tap'][p]**2)*(1/(Dados_linha['R (pu)'][p] + 1j*(Dados_linha['X (pu)'][p])))
            else:
              Ybarra[k-1][m-1] = Ybarra[k-1][m-1] + (1/(Dados_linha['R (pu)'][p] + 1j*(Dados_linha['X (pu)'][p])))
      else:
        for p in Dados_linha.index:
          if (Dados_linha['De'][p]==k and Dados_linha['Para'][p]==m):
            Ybarra[k-1][m-1] = Ybarra[k-1][m-1] - (Dados_linha['Tap'][p]/((Dados_linha['R (pu)'][p] + 1j*(Dados_linha['X (pu)'][p]))))*np.exp(-1j*Dados_linha['Defasagem (°)'][p]*np.pi/180)
          elif (Dados_linha['De'][p]==m and Dados_linha['Para'][p]==k):
            Ybarra[k-1][m-1] = Ybarra[k-1][m-1] - (Dados_linha['Tap'][p]/((Dados_linha['R (pu)'][p] + 1j*(Dados_linha['X (pu)'][p]))))*np.exp(1j*Dados_linha['Defasagem (°)'][p]*np.pi/180)

  # Acréscimo dos elementos shunts de linhas

  for p in Dados_linha.index:
    Ybarra[int(Dados_linha['De'][p])-1][int(Dados_linha['De'][p])-1] = Ybarra[int(Dados_linha['De'][p])-1][int(Dados_linha['De'][p])-1] + 1j*Dados_linha['B/2 (pu)'][p]
    Ybarra[int(Dados_linha['Para'][p])-1][int(Dados_linha['Para'][p])-1] = Ybarra[int(Dados_linha['Para'][p])-1][int(Dados_linha['Para'][p])-1] + 1j*Dados_linha['B/2 (pu)'][p]

  # Acréscimo dos elementos shunts de barra

  for p in Dados_barra.index:
    Ybarra[int(p)-1][int(p)-1] = Ybarra[int(p)-1][int(p)-1] + Dados_barra['Shunt (pu)'][p]

  G = np.real(Ybarra)
  B = np.imag(Ybarra)

  return G,B

In [ ]:
Ybus = calculo_ybus(Dados_barra,Dados_linha)

G = Ybus[0] # matriz condutância
B = Ybus[1] # matriz susceptância

## Função - Cálculo do fluxo de potência via Newton Raphson

O cálculo será feito considerando as matrizes completas:

\begin{equation}
\begin{bmatrix}
\Delta P_1 \\ \Delta P_2 \\ \vdots \\ \Delta P_n \\ \Delta Q_1 \\ \Delta Q_2 \\ \vdots \\ \Delta Q_n
\end{bmatrix} =
J\cdot
\begin{bmatrix}
\Delta \theta_1 \\ \Delta \theta_2 \\ \vdots \\ \Delta \theta_n \\ \Delta V_1 \\ \Delta V_2 \\ \vdots \\ \Delta V_n
\end{bmatrix}
\end{equation}

Assim, na diagonal da matriz Jacobiana, haverá o acréscimo do big number para aquelas variáveis que não precisam ou não podem ser calculadas no sistema linear a exemplo de:

- Ângulo e módulo da tensão para a barra slack;
- Módulo de tensão para barra PV.

In [ ]:
def fluxo_potencia(B,G,Dados_barra,Dados_linha):

  #==============================================================
  #=================== Inicialização das variáveis ==============
  #==============================================================

  #------------------- Valores especificados --------------------

  P_esp = dict()
  Q_esp = dict()

  #------------------- Valores calculados -----------------------

  V = dict()

  Theta = dict()

  #------------------- Inicialização ----------------------------

  for k in Dados_barra.index:
    if Dados_barra['Tipo'][k]=='PV':
      P_esp[str(k)] = float(Dados_barra['Pg (pu)'][k]) - float(Dados_barra['Pd (pu)'][k])
      Q_esp[str(k)] = 0.0 # Valor qualquer pois será calculado para a barra PV
      V[str(k)] = float(Dados_barra['V (pu)'][k])
      Theta[str(k)] = 0.0 # valor inicial pois será calculado para a barra PV
    elif Dados_barra['Tipo'][k]=='PQ':
      P_esp[str(k)] = float(Dados_barra['Pg (pu)'][k]) - float(Dados_barra['Pd (pu)'][k])
      Q_esp[str(k)] = float(Dados_barra['Qg (pu)'][k]) - float(Dados_barra['Qd (pu)'][k])
      V[str(k)] = 1.0 # valor inicial pois será calculado para a barra PQ
      Theta[str(k)] = 0.0 # valor inicial pois será calculado para a barra PQ
    else:
      P_esp[str(k)] = 0.0 # Valor qualquer pois será calculado para a barra Slack
      Q_esp[str(k)] = 0.0 # Valor qualquer pois será calculado para a barra Slack
      V[str(k)] = float(Dados_barra['V (pu)'][k])
      Theta[str(k)] = float(Dados_barra['Ângulo (°)'][k])*np.pi/180

  #==================================================================
  #=================== Fim inicialização das variáveis ==============
  #==================================================================

  B = B.copy()
  G = G.copy()
  Dados_barra = Dados_barra.copy()
  Dados_linha = Dados_linha.copy()

  #--------------------------- Cálculo do chute inicial --------------------------

  Delta_Y = np.zeros((2*np.size(Dados_barra,0),1))

  P_calc = np.zeros((np.size(Dados_barra,0),1))

  Q_calc = np.zeros((np.size(Dados_barra,0),1))

  for k in Dados_barra.index:
    if Dados_barra['Tipo'][k]=='PQ':
      for m in Dados_barra.index:
        P_calc[k-1][0] = P_calc[k-1][0] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
        Q_calc[k-1][0] = Q_calc[k-1][0] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))
    elif Dados_barra['Tipo'][k]=='PV':
      for m in Dados_barra.index:
        P_calc[k-1][0] = P_calc[k-1][0] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
        Q_calc[k-1][0] = 1 # não é calculado à cada iteração nas barras PV
    else:
      P_calc[k-1][0] = 1 # não é calculado à cada iteração na barra Slack
      Q_calc[k-1][0] = 1 # não é calculado à cada iteração na barra Slack

  #---------------------------------------- Cálculo dos resíduos -----------------------------------------------

  for k in Dados_barra.index:
    if Dados_barra['Tipo'][k]=='Slack':
      Delta_Y[k-1][0] = 0
      Delta_Y[k+np.size(Dados_barra,0)-1][0] = 0
    elif Dados_barra['Tipo'][k]=='PV':
      Delta_Y[k-1][0] = P_esp[str(k)] - P_calc[k-1][0]
      Delta_Y[k+np.size(Dados_barra,0)-1][0] = 0
    else:
      Delta_Y[k-1][0] = P_esp[str(k)] - P_calc[k-1][0]
      Delta_Y[k+np.size(Dados_barra,0)-1][0] = Q_esp[str(k)] - Q_calc[k-1][0]

  #--------------------------------------- Define parâmetros do método de Newton Raphson --------------------

  iteracao = 0
  tolerancia = 1e-5

  #==========================================================================================================
  #==========================================================================================================
  #======================================= Início do processo iterativo =====================================
  #==========================================================================================================
  #==========================================================================================================

  while np.max(np.abs(Delta_Y))>tolerancia:

    #---------------- Cálculo da Matriz Jacobiana --------------------

    #-------- Submatriz H (Derivada de P em relação a Theta) ---------

    H = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)))

    for k in Dados_barra.index:
      for m in Dados_barra.index:
        if k==m:
          for p in Dados_barra.index:
            H[k-1][m-1] = H[k-1][m-1] + V[str(k)]*V[str(p)]*(-G[k-1][p-1]*np.sin(Theta[str(k)] - Theta[str(p)]) + B[k-1][p-1]*np.cos(Theta[str(k)] - Theta[str(p)]))
          H[k-1][m-1] = H[k-1][m-1] - B[k-1][m-1]*(V[str(k)]**2)
        else:
          H[k-1][m-1] = H[k-1][m-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))

    #-------- Submatriz N (Derivada de P em relação a V) ---------

    N = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)))

    for k in Dados_barra.index:
      for m in Dados_barra.index:
        if k==m:
          for p in Dados_barra.index:
            N[k-1][m-1] = N[k-1][m-1] + V[str(p)]*(G[k-1][p-1]*np.cos(Theta[str(k)] - Theta[str(p)]) + B[k-1][p-1]*np.sin(Theta[str(k)] - Theta[str(p)]))
          N[k-1][m-1] = N[k-1][m-1] + G[k-1][m-1]*(V[str(k)])
        else:
          N[k-1][m-1] = N[k-1][m-1] + V[str(k)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))

    #-------- Submatriz M (Derivada de Q em relação a Theta) ---------

    M = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)))

    for k in Dados_barra.index:
      for m in Dados_barra.index:
        if k==m:
          for p in Dados_barra.index:
            M[k-1][m-1] = M[k-1][m-1] + V[str(k)]*V[str(p)]*(G[k-1][p-1]*np.cos(Theta[str(k)] - Theta[str(p)]) + B[k-1][p-1]*np.sin(Theta[str(k)] - Theta[str(p)]))
          M[k-1][m-1] = M[k-1][m-1] - G[k-1][m-1]*(V[str(k)]**2)
        else:
          M[k-1][m-1] = M[k-1][m-1] + V[str(k)]*V[str(m)]*(-G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))

    #-------- Submatriz L (Derivada de Q em relação a V) ---------

    L = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)))

    for k in Dados_barra.index:
      for m in Dados_barra.index:
        if k==m:
          for p in Dados_barra.index:
            L[k-1][m-1] = L[k-1][m-1] + V[str(p)]*(G[k-1][p-1]*np.sin(Theta[str(k)] - Theta[str(p)]) - B[k-1][p-1]*np.cos(Theta[str(k)] - Theta[str(p)]))
          L[k-1][m-1] = L[k-1][m-1] - B[k-1][m-1]*(V[str(k)])
        else:
          L[k-1][m-1] = L[k-1][m-1] + V[str(k)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))

    #-------------------------- Junção das submatrizes ----------------------------------

    Derivada_P = np.concatenate((H,N),axis = 1)
    Derivada_Q = np.concatenate((M,L),axis = 1)

    Jacobiana = np.concatenate((Derivada_P,Derivada_Q),axis = 0)

    # Eliminando as linhas e colunas de equações não necessárias

    for k in Dados_barra.index:
      if Dados_barra['Tipo'][k]=='Slack':
        Jacobiana[k-1][k-1] = 1e100
        Jacobiana[k-1 + np.size(Dados_barra,0)][k-1 + np.size(Dados_barra,0)] = 1e100
      elif Dados_barra['Tipo'][k]=='PV':
        Jacobiana[k-1 + np.size(Dados_barra,0)][k-1 + np.size(Dados_barra,0)] = 1e100

    #------------------------- Resolução do sistema linear ------------------------------

    Delta_X = np.linalg.solve(Jacobiana,Delta_Y)

    #------------------------- Atualização das variáveis calculadas ---------------------

    for k in Dados_barra.index:
      Theta[str(k)] = Theta[str(k)] + Delta_X[k-1][0]
      V[str(k)] = V[str(k)] + Delta_X[k-1 + np.size(Dados_barra,0)][0]

    #------------------ Cálculo dos resíduos frente aos valores especificados -----------


    Delta_Y = np.zeros((2*np.size(Dados_barra,0),1))

    P_calc = np.zeros((np.size(Dados_barra,0),1))

    Q_calc = np.zeros((np.size(Dados_barra,0),1))

    for k in Dados_barra.index:
      if Dados_barra['Tipo'][k]=='PQ':
        for m in Dados_barra.index:
          P_calc[k-1][0] = P_calc[k-1][0] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
          Q_calc[k-1][0] = Q_calc[k-1][0] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))
      elif Dados_barra['Tipo'][k]=='PV':
        for m in Dados_barra.index:
          P_calc[k-1][0] = P_calc[k-1][0] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
          Q_calc[k-1][0] = 0 # não é calculado à cada iteração nas barras PV
      else:
        P_calc[k-1][0] = 0 # não é calculado à cada iteração na barra Slack
        Q_calc[k-1][0] = 0 # não é calculado à cada iteração na barra Slack

    #---------------------------------------- Cálculo dos resíduos -----------------------------------------------

    for k in Dados_barra.index:
      if Dados_barra['Tipo'][k]=='Slack':
        Delta_Y[k-1][0] = 0
        Delta_Y[k+np.size(Dados_barra,0)-1][0] = 0
      elif Dados_barra['Tipo'][k]=='PV':
        Delta_Y[k-1][0] = P_esp[str(k)] - P_calc[k-1][0]
        Delta_Y[k+np.size(Dados_barra,0)-1][0] = 0
      else:
        Delta_Y[k-1][0] = P_esp[str(k)] - P_calc[k-1][0]
        Delta_Y[k+np.size(Dados_barra,0)-1][0] = Q_esp[str(k)] - Q_calc[k-1][0]

    iteracao = iteracao + 1

  #==========================================================================================================
  #==========================================================================================================
  #========================================= Fim do processo iterativo ======================================
  #==========================================================================================================
  #==========================================================================================================

  #==================================== Cálculo do estado operativo da rede ==================================

  #=========================================== Barras ========================================================

  v = np.zeros((1,np.size(Dados_barra,0)))
  th = np.zeros((1,np.size(Dados_barra,0)))
  th2grau = np.zeros((1,np.size(Dados_barra,0)))
  Pk_calc = np.zeros((1,np.size(Dados_barra,0)))
  Qk_calc = np.zeros((1,np.size(Dados_barra,0)))

  for k in Dados_barra.index:
    v[0][k-1] = V[str(k)]
    th[0][k-1] = Theta[str(k)]
    th2grau[0][k-1] = Theta[str(k)]*180/np.pi

  for k in Dados_barra.index:
    if Dados_barra['Tipo'][k]=='PQ':
      for m in Dados_barra.index:
        Pk_calc[0][k-1] = Pk_calc[0][k-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
        Qk_calc[0][k-1] = Qk_calc[0][k-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))
        #Pk_calc[0][k-1] = float(Dados_barra['Pg (pu)'][k]) - float(Dados_barra['Pd (pu)'][k])
        #Qk_calc[0][k-1] = float(Dados_barra['Qg (pu)'][k]) - float(Dados_barra['Qd (pu)'][k])
    elif Dados_barra['Tipo'][k]=='PV':
      for m in Dados_barra.index:
        Pk_calc[0][k-1] = Pk_calc[0][k-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
        Qk_calc[0][k-1] = Qk_calc[0][k-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))
    else:
      for m in Dados_barra.index:
        Pk_calc[0][k-1] = Pk_calc[0][k-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]) + B[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]))
        Qk_calc[0][k-1] = Qk_calc[0][k-1] + V[str(k)]*V[str(m)]*(G[k-1][m-1]*np.sin(Theta[str(k)] - Theta[str(m)]) - B[k-1][m-1]*np.cos(Theta[str(k)] - Theta[str(m)]))

  Estado_barra = pd.DataFrame(data = np.transpose(np.concatenate(([numero_barra],v,th,th2grau,Pk_calc,Qk_calc),axis = 0)),columns=['Barra','V (pu)','Ângulo (rad)','Ângulo (°)','Pk (pu)','Qk (pu)'])

  Estado_barra.set_index('Barra',inplace=True)

  Estado_barra.index = Estado_barra.index.astype('int')

  Estado_barra


  #=========================================== Linhas ========================================================

  Pkm = np.zeros((np.size(Dados_linha,0),2))
  Qkm = np.zeros((np.size(Dados_linha,0),2))
  Ikm = np.zeros((np.size(Dados_linha,0),2),dtype='complex_')

  for k in Dados_linha.index:
    de = int(Dados_linha['De'][k])
    para = int(Dados_linha['Para'][k])
    #-------------------------------- Potência ativa k-m------------------------------------------------
    Pkm[k][0] = (((Dados_linha['Tap'][k])**2)*np.real(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]**2) - ((Dados_linha['Tap'][k])*np.imag(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.sin(Theta[str(de)] - Theta[str(para)] + (Dados_linha['Defasagem (°)'][k]*np.pi/180))) - ((Dados_linha['Tap'][k])*np.real(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.cos(Theta[str(de)] - Theta[str(para)] + (Dados_linha['Defasagem (°)'][k]*np.pi/180)))
    Pkm[k][1] = (np.real(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(para)]**2) - (Dados_linha['Tap'][k]*np.imag(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.sin(Theta[str(para)] - Theta[str(de)] - (Dados_linha['Defasagem (°)'][k]*np.pi/180))) - (Dados_linha['Tap'][k]*np.real(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.cos(Theta[str(para)] - Theta[str(de)] - (Dados_linha['Defasagem (°)'][k]*np.pi/180)))
    #-------------------------------- Potência reativa k-m------------------------------------------------
    Qkm[k][0] = ((-Dados_linha['Tap'][k]**2)*np.imag(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]**2) - (Dados_linha['B/2 (pu)'][k]*V[str(de)]**2) - (Dados_linha['Tap'][k]*np.real(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.sin(Theta[str(de)] - Theta[str(para)] + (Dados_linha['Defasagem (°)'][k]*np.pi/180))) + (Dados_linha['Tap'][k]*np.imag(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.cos(Theta[str(de)] - Theta[str(para)] + (Dados_linha['Defasagem (°)'][k]*np.pi/180)))
    Qkm[k][1] = (-np.imag(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(para)]**2) - (Dados_linha['B/2 (pu)'][k]*V[str(para)]**2) - (Dados_linha['Tap'][k]*np.real(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.sin(Theta[str(para)] - Theta[str(de)] - (Dados_linha['Defasagem (°)'][k]*np.pi/180))) + (Dados_linha['Tap'][k]*np.imag(1/(Dados_linha['R (pu)'][k] + 1j*Dados_linha['X (pu)'][k]))*V[str(de)]*V[str(para)]*np.cos(Theta[str(para)] - Theta[str(de)] - (Dados_linha['Defasagem (°)'][k]*np.pi/180)))
    #------------------------------- Corrente I k-m ------------------------------------------------------
    Ikm[k][0] = np.conjugate((Pkm[k][0] + 1j*Qkm[k][0])/(Estado_barra['V (pu)'][int(de)]*np.exp(1j*Estado_barra['Ângulo (rad)'][int(de)]))) #- 1j*Estado_barra['V (pu)'][int(de)]*np.exp(1j*Estado_barra['Ângulo (rad)'][int(de)])*Dados_linha['B/2 (pu)'][k]
    Ikm[k][1] = np.conjugate((Pkm[k][1] + 1j*Qkm[k][1])/(Estado_barra['V (pu)'][int(para)]*np.exp(1j*Estado_barra['Ângulo (rad)'][int(para)]))) #- 1j*Estado_barra['V (pu)'][int(para)]*np.exp(1j*Estado_barra['Ângulo (rad)'][int(para)])*Dados_linha['B/2 (pu)'][k]

  data = np.zeros((2*np.size(Dados_linha,0),7))

  for k in Dados_linha.index:
    data[2*k][0] = int(Dados_linha['De'][k])
    data[2*k][1] = int(Dados_linha['Para'][k])
    data[2*k][2] = Pkm[k][0]
    data[2*k][3] = Qkm[k][0]
    data[2*k][4] = np.abs(Ikm[k][0])
    data[2*k][5] = np.angle(Ikm[k][0])
    data[2*k][6] = np.angle(Ikm[k][0])*180/np.pi
    data[2*k + 1][0] = int(Dados_linha['Para'][k])
    data[2*k + 1][1] = int(Dados_linha['De'][k])
    data[2*k + 1][2] = Pkm[k][1]
    data[2*k + 1][3] = Qkm[k][1]
    data[2*k + 1][4] = np.abs(Ikm[k][1])
    data[2*k + 1][5] = np.angle(Ikm[k][1])
    data[2*k + 1][6] = np.angle(Ikm[k][1])*180/np.pi

  Fluxos = pd.DataFrame(data = data,columns=['De','Para','Pkm (pu)','Qkm (pu)','|Ikm| (pu)','Fase Ikm (rad)','Fase Ikm (°)'])

  Fluxos

  #========================================== Return da função ============================================

  return Estado_barra,Fluxos


## Cálculo do estado operativo da rede

In [ ]:
Pot = fluxo_potencia(B,G,Dados_barra,Dados_linha) # função que dá o estado operativo da rede na frequência fundamental

In [ ]:
Estado_barra = Pot[0]
Fluxos = Pot[1]

- Barras

In [ ]:
Estado_barra

,V (pu),Ângulo (rad),Ângulo (°),Pk (pu),Qk (pu)
Barra,,,,,
1,1.000000,-7.363168e-101,-4.218784e-99,7.359523e-01,5.359363e-02
2,0.995000,-4.117595e-02,-2.359208e+00,2.000024e-01,1.859626e-01
3,0.998821,-2.106410e-03,-1.206884e-01,-1.664207e-06,6.029417e-06
4,0.996762,-6.013180e-02,-3.445298e+00,-5.999995e-02,-5.299990e-02
5,0.994510,-4.135782e-02,-2.369628e+00,-2.239962e-01,-1.999936e-01
6,0.994223,-4.139661e-02,-2.371851e+00,-1.129304e-06,1.345340e-06
7,1.002284,-7.966840e-02,-4.564663e+00,-1.149993e-01,-2.899807e-02
8,1.001176,-7.990496e-02,-4.578217e+00,-1.309991e-01,-1.129977e-01
9,0.994269,-4.133109e-02,-2.368097e+00,-1.095256e-06,8.305043e-07


- Linhas

In [ ]:
Fluxos

,De,Para,Pkm (pu),Qkm (pu),|Ikm| (pu),Fase Ikm (rad),Fase Ikm (°)
0,1.0,3.0,0.735952,0.053594,0.737901,-0.072694,-4.165050
1,3.0,1.0,-0.735195,-0.051982,0.737901,3.068899,175.834950
2,5.0,2.0,-0.139545,-0.130435,0.192068,2.348567,134.562968
3,2.0,5.0,0.139590,0.130525,0.192068,-0.793026,-45.437032
4,5.0,6.0,0.248182,0.157327,0.295469,-0.606345,-34.741006
5,6.0,5.0,-0.248116,-0.157272,0.295469,2.535248,145.258994
6,5.0,9.0,0.081802,0.085086,0.118682,-0.846432,-48.496976
7,9.0,5.0,-0.081780,-0.085067,0.118682,2.295161,131.503024
8,5.0,11.0,0.319054,0.304457,0.443444,-0.803350,-46.028545
9,11.0,5.0,-0.318839,-0.304278,0.443444,2.338243,133.971455


# **Análise harmônica em sistemas de potência**

## Dados da fonte harmônica

 - Espectro advindo de algum modelo já conhecido.

In [ ]:
qtd_fontes = 1

bus_harmonica = [7] # número da(s) barra(s) onde está a fonte harmônica

enderecos=list()

for k in bus_harmonica:
  enderecos.append('Bus('+str(k)+')') # endereços para identificação das barras com fonte harmônica

#--------------------------------------------------------------------------

ordem_harmonica = dict() # dicionário cujos índices são as barras com fonte harmônica
for k in enderecos:
  ordem_harmonica[k] = []

# entrada manual das ordens harmônicas consideradas na fonte

#ordem_harmonica['Bus(4)'] = [1,5,7,11,13,17,19,23,25,29,31,35,37] # ASD
ordem_harmonica['Bus(7)'] = [1,5,7,11,13,17,19,23,25,29,31,35,37] # TCR
#ordem_harmonica['Bus(8)'] = [1,5,7,11,13,17,19,23,25,29,31,35,37] # Lâmpada fluorescente
#ordem_harmonica['Bus(10)'] = [1,5,7,11,13,17,19,23,25,29,31,35,37] # Lâmpada fluorescente
#ordem_harmonica['Bus(12)'] = [1,5,7,11,13,17,19,23,25,29,31,35,37] # Lâmpada fluorescente
#ordem_harmonica['Bus(13)'] = [1,5,7,11,13,17,19,23,25,29,31,35,37] # Lâmpada fluorescente
#--------------------------------------------------------------------------

#Freq = {'Bus(4)':[]}
Freq = {'Bus(7)':[]}
#Freq = {'Bus(8)':[]}
#Freq = {'Bus(10)':[]}
#Freq = {'Bus(12)':[]}
#Freq = {'Bus(13)':[]}

#Freq['Bus(4)'] = [60*i for i in ordem_harmonica['Bus(4)']]
Freq['Bus(7)'] = [60*i for i in ordem_harmonica['Bus(7)']]
#Freq['Bus(8)'] = [60*i for i in ordem_harmonica['Bus(8)']]
#Freq['Bus(10)'] = [60*i for i in ordem_harmonica['Bus(10)']]
#Freq['Bus(12)'] = [60*i for i in ordem_harmonica['Bus(12)']]
#Freq['Bus(13)'] = [60*i for i in ordem_harmonica['Bus(12)']]

#--------------------------------------------------------------------------

#Magnitude = {'Bus(4)':[]}
Magnitude = {'Bus(7)':[]}
#Magnitude = {'Bus(8)':[]}
#Magnitude = {'Bus(10)':[]}
#Magnitude = {'Bus(12)':[]}
#Magnitude = {'Bus(13)':[]}

# entrada manual das magnitudes em % das ordens

#Magnitude['Bus(4)'] = [100,2*7.02,2*2.50,2*1.36,2*0.75,2*0.62,2*0.32,2*0.43,2*0.13,2*0.40,0,0,0] # TCR
Magnitude['Bus(7)'] = [100,2*18.24,2*11.90,2*5.73,2*4.01,2*1.93,2*1.39,2*0.94,2*0.86,2*0.71,2*0.62,2*0.44,2*0.38] # ASD
#Magnitude['Bus(8)'] = [100,10.7,2.1,0.9,0.6,0,0,0,0,0.0,0.0,0.0,0.0] # Lâmpada fluorescente
#Magnitude['Bus(10)'] = [100,10.7,2.1,0.9,0.6,0,0,0,0,0.0,0.0,0.0,0.0]  # Lâmpada fluorescente
#Magnitude['Bus(12)'] = [100,10.7,2.1,0.9,0.6,0,0,0,0,0.0,0.0,0.0,0.0]  # Lâmpada fluorescente
#Magnitude['Bus(13)'] = [100,10.7,2.1,0.9,0.6,0,0,0,0,0.0,0.0,0.0,0.0]  # Lâmpada fluorescente
#--------------------------------------------------------------------------

#Fase_h_grau = {'Bus(4)':[]}
Fase_h_grau = {'Bus(7)':[]}
#Fase_h_grau = {'Bus(8)':[]}
#Fase_h_grau = {'Bus(10)':[]}
#Fase_h_grau = {'Bus(12)':[]}
#Fase_h_grau = {'Bus(13)':[]}

#Fase_h_grau['Bus(4)'] = [46.92,-124.40,-29.87,-23.75,71.50,77.12,173.43,178.02,-83.45,-80.45,0,0,0] # TCR
Fase_h_grau['Bus(7)'] = [0,-55.68,-84.11,-143.56,-175.58,111.39,68.30,-24.61,-67.64,-145.46,176.83,97.40,54.36] # ASD
#Fase_h_grau['Bus(8)'] = [-41.2,339.0,137.7,39.8,182.4,0,0,0,0,0.0,0.0,0.0,0.0] # Lâmpada fluorescente
#Fase_h_grau['Bus(10)'] = [-41.2,339.0,137.7,39.8,182.4,0,0,0,0,0.0,0.0,0.0,0.0] # Lâmpada fluorescente
#Fase_h_grau['Bus(12)'] = [-41.2,339.0,137.7,39.8,182.4,0,0,0,0,0.0,0.0,0.0,0.0] # Lâmpada fluorescente
#Fase_h_grau['Bus(13)'] = [-41.2,339.0,137.7,39.8,182.4,0,0,0,0,0.0,0.0,0.0,0.0] # Lâmpada fluorescente

# entrada manual das fases em (°) das ordens harmônicas consideradas na fonte

#--------------------------------------------------------------------------

#Espectro = {'Bus(4)':0}
Espectro = {'Bus(7)':0}
#Espectro = {'Bus(8)':0}
#Espectro = {'Bus(10)':0}
#Espectro = {'Bus(12)':0}
#Espectro = {'Bus(13)':0}

for k in enderecos:
  Espectro[k] = pd.DataFrame(data = np.transpose([ordem_harmonica[k],Freq[k],Magnitude[k],Fase_h_grau[k]]), columns=['Ordem harmônica','Frequência (Hz)','Magnitude (%)','Fase (°)'])
  Espectro[k].set_index('Ordem harmônica',inplace=True)

 - Espectro da fonte baseado nos dados passados acima

In [ ]:
Magnitude_fonte = dict()
Fase_fonte = dict()
Espectro_fonte = dict()

for k in bus_harmonica:
  Magnitude_fonte['Bus('+str(k)+')'] = list()
  Fase_fonte['Bus('+str(k)+')'] = list()
  Espectro_fonte['Bus('+str(k)+')'] = 0

  I_fund_busk_inj = np.conjugate((Estado_barra['Pk (pu)'][k] + 1j*Estado_barra['Qk (pu)'][k])/(Estado_barra['V (pu)'][k]*np.exp(1j*Estado_barra['Ângulo (rad)'][k])))

  for s in Espectro['Bus('+str(k)+')'].index:

    Magnitude_fonte['Bus('+str(k)+')'].append(np.abs(I_fund_busk_inj)*Espectro['Bus('+str(k)+')']['Magnitude (%)'][s]/100)
    Fase_fonte['Bus('+str(k)+')'].append(Espectro['Bus('+str(k)+')']['Fase (°)'][s] + s*((np.angle(I_fund_busk_inj)*180/np.pi) - Espectro['Bus('+str(k)+')']['Fase (°)'][1]))

  Espectro_fonte['Bus('+str(k)+')'] = pd.DataFrame(data = np.transpose([ordem_harmonica['Bus('+str(k)+')'],Freq['Bus('+str(k)+')'],Magnitude_fonte['Bus('+str(k)+')'],Fase_fonte['Bus('+str(k)+')']]), columns=['Ordem harmônica','Frequência (Hz)','Magnitude (p.u)','Fase (°)'])
  Espectro_fonte['Bus('+str(k)+')'].set_index('Ordem harmônica',inplace=True)

## Ybarra para cada ordem harmônica

### Matriz Y referente aos elementos shunts

 - Frequência fundamental

In [ ]:
y_shunt_fund = dict()

for k in Estado_barra.index:
  if Dados_barra['Tipo'][k]=='PQ':
      y_shunt_fund['Bus('+str(k)+')'] = np.conjugate((-Estado_barra['Pk (pu)'][k]-1j*Estado_barra['Qk (pu)'][k])/(Estado_barra['V (pu)'][k]**2))
  elif Dados_barra['Tipo'][k]=='PV':
      y_shunt_fund['Bus('+str(k)+')'] = 0 + 1j*0
  else:
      y_shunt_fund['Bus('+str(k)+')'] = 0.0

 - Demais ordens harmônicas

In [ ]:
# Determinação de todas ordens harmônicas a serem trabalhadas

ordens_harmonicas = list()

for k in list(Espectro_fonte.keys()):
  for s in Espectro_fonte[k].index:
    if s not in ordens_harmonicas:
      ordens_harmonicas.append(int(s))

ordens_harmonicas

[1, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]

In [ ]:
Yshunt_h = dict()

for h in ordens_harmonicas:

   Yshunt_h[str(h)] = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)),dtype='complex_')

   for k in Dados_barra.index:
      if np.imag(y_shunt_fund['Bus('+str(k)+')'])<0:
        Yshunt_h[str(h)][k-1][k-1] = Yshunt_h[str(h)][k-1][k-1] + np.real(y_shunt_fund['Bus('+str(k)+')']) + 1j*(np.imag(y_shunt_fund['Bus('+str(k)+')'])/h)
      else:
        Yshunt_h[str(h)][k-1][k-1] = Yshunt_h[str(h)][k-1][k-1] + np.real(y_shunt_fund['Bus('+str(k)+')']) + 1j*(np.imag(y_shunt_fund['Bus('+str(k)+')'])*h)

In [ ]:
Yshunt_h

{'1': array([[0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
        [0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
        [0.00000000e+00+0.00000000e

### Matriz de admitância nodal

In [ ]:
Ynodal_h = dict() # dicionário cujos endereços são as ordens harmônicas

In [ ]:
for h in ordens_harmonicas:

  Ynodal_h[str(h)] = np.zeros((np.size(Dados_barra,0),np.size(Dados_barra,0)),dtype='complex_')

  for k in Dados_barra.index:
    for m in Dados_barra.index:
      if k==m:
        for p in Dados_linha.index:
          if Dados_linha['De'][p]==k or Dados_linha['Para'][p]==k:
            if Dados_linha['De'][p]==k:
               Ynodal_h[str(h)][k-1][m-1] =  Ynodal_h[str(h)][k-1][m-1] + (Dados_linha['Tap'][p]**2)*(1/(Dados_linha['R (pu)'][p] + 1j*(h*Dados_linha['X (pu)'][p])))
            else:
               Ynodal_h[str(h)][k-1][m-1] =  Ynodal_h[str(h)][k-1][m-1] + (1/(Dados_linha['R (pu)'][p] + 1j*(h*Dados_linha['X (pu)'][p])))
      else:
        for p in Dados_linha.index:
          if (Dados_linha['De'][p]==k and Dados_linha['Para'][p]==m):
             Ynodal_h[str(h)][k-1][m-1] =  Ynodal_h[str(h)][k-1][m-1] - (Dados_linha['Tap'][p]/((Dados_linha['R (pu)'][p] + 1j*(h*Dados_linha['X (pu)'][p]))))*np.exp(-1j*Dados_linha['Defasagem (°)'][p]*np.pi/180)
          elif (Dados_linha['De'][p]==m and Dados_linha['Para'][p]==k):
             Ynodal_h[str(h)][k-1][m-1] =  Ynodal_h[str(h)][k-1][m-1] - (Dados_linha['Tap'][p]/((Dados_linha['R (pu)'][p] + 1j*(h*Dados_linha['X (pu)'][p]))))*np.exp(1j*Dados_linha['Defasagem (°)'][p]*np.pi/180)

      if Dados_barra['Tipo'][k]=='Slack':
          Ynodal_h[str(h)][k-1][k-1] = 10**20

  # Acréscimo dos elementos shunts de linhas

  for p in Dados_linha.index:
     Ynodal_h[str(h)][int(Dados_linha['De'][p])-1][int(Dados_linha['De'][p])-1] =  Ynodal_h[str(h)][int(Dados_linha['De'][p])-1][int(Dados_linha['De'][p])-1] + 1j*h*Dados_linha['B/2 (pu)'][p]
     Ynodal_h[str(h)][int(Dados_linha['Para'][p])-1][int(Dados_linha['Para'][p])-1] =  Ynodal_h[str(h)][int(Dados_linha['Para'][p])-1][int(Dados_linha['Para'][p])-1] + 1j*h*Dados_linha['B/2 (pu)'][p]

  # Acréscimo dos elementos shunts de barra

  for p in Dados_barra.index:
    if np.imag(Dados_barra['Shunt (pu)'][p])<0:
      Ynodal_h[str(h)][int(p)-1][int(p)-1] = Ynodal_h[str(h)][int(p)-1][int(p)-1] + np.real(Dados_barra['Shunt (pu)'][p]) + 1j*np.imag(Dados_barra['Shunt (pu)'][p])/h
    else:
      Ynodal_h[str(h)][int(p)-1][int(p)-1] = Ynodal_h[str(h)][int(p)-1][int(p)-1] + np.real(Dados_barra['Shunt (pu)'][p]) + 1j*np.imag(Dados_barra['Shunt (pu)'][p])*h

In [ ]:
Ynodal_h

{'1': array([[ 1.00000000e+20   +0.j        ,  0.00000000e+00   +0.j        ,
         -1.29983074e+02 +276.79848883j,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ],
        [ 0.00000000e+00   +0.j        ,  1.65472439e+02 -331.38371114j,
          0.00000000e+00   +0.j        , -4.46548615e-01   +2.639073j  ,
         -1.65014270e+02 +328.67596337j,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ,  0.00000000e+00   +0.j        ,
          0.00000000e+00   +0.j        ],
        [-1.29983074e+02 +276.79848

## Função - Método da compensação das correntes

In [ ]:
def fluxo_harmonico(Ynodal_h,Yshunt_h,Espectro_fonte,ordens_harmonicas,bus_harmonica,Dados_barra,Estado_barra):

  Ynodal_h_copy = Ynodal_h.copy()
  Yshunt_h_copy = Yshunt_h.copy()
  Espectro_fonte_copy = Espectro_fonte.copy()
  Dados_barra_copy = Dados_barra.copy()
  Estado_barra_copy = Estado_barra.copy()

  Vk_h = dict()
  Ik_h = dict()

  for h in ordens_harmonicas:

    #-------------- Inicialização --------------------------------

    Vk_h[str(h)] = np.zeros((np.size(Ynodal_h_copy[str(h)],0),1),dtype='complex')

    I_injecao = np.zeros((np.size(Ynodal_h_copy[str(h)],0),1),dtype='complex')

    for k in bus_harmonica:

      for s in Espectro_fonte_copy['Bus('+str(k)+')'].index:

        if s==h:

          I_injecao[k-1][0] = I_injecao[k-1][0] + Espectro_fonte_copy['Bus('+str(k)+')']['Magnitude (p.u)'][s]*np.exp(1j*(Espectro_fonte_copy['Bus('+str(k)+')']['Fase (°)'][s]*np.pi/180))

    if h==1:
      for k in Dados_barra_copy.index:
        if Dados_barra_copy['Tipo'][k]=='PV':
          I_injecao[k-1][0] = I_injecao[k-1][0] + np.conjugate((Estado_barra_copy['Pk (pu)'][k] + 1j*Estado_barra_copy['Qk (pu)'][k])/(Estado_barra_copy['V (pu)'][k]*np.exp(1j*Estado_barra_copy['Ângulo (rad)'][k])))

    # -> Cálculo da tensão em todas as barras para tal ordem harmônica

    V = np.dot(np.linalg.inv(Ynodal_h_copy[str(h)] + Yshunt_h_copy[str(h)]),I_injecao)

    tol = 1e-5

    max_residuo = np.max(np.abs(np.subtract(V,Vk_h[str(h)])))

    #----------------------- Processo iterativo ------------------------------

    iteracao = 0

    while max_residuo>tol and iteracao<20:

      I_absorvida = np.dot(Yshunt_h_copy[str(h)],V)

      Vk_h[str(h)] = np.dot(np.linalg.inv(Ynodal_h_copy[str(h)] + Yshunt_h_copy[str(h)]),np.subtract(I_injecao,I_absorvida))

      max_residuo = np.max(np.abs(np.subtract(V,Vk_h[str(h)])))

      V = Vk_h[str(h)]

      iteracao = iteracao + 1

    Ik_h[str(h)] = np.dot(np.add(Ynodal_h_copy[str(h)],Yshunt_h_copy[str(h)]),Vk_h[str(h)])

  #----------------------- Cálculo THD por barra ---------------------------

  THDv_bus = dict()

  for bus in Dados_barra_copy.index:

    if Dados_barra_copy['Tipo'][bus]=='Slack':

      THDv_bus[str(bus)] = 0

    else:

      THDv_bus[str(bus)] = 0

      for u in ordens_harmonicas:

        if u!=1:

          THDv_bus[str(bus)] = THDv_bus[str(bus)] + np.abs(Vk_h[str(u)][bus-1][0])**2

      THDv_bus[str(bus)] = np.sqrt(THDv_bus[str(bus)])/(Estado_barra_copy['V (pu)'][bus])


  return Vk_h,THDv_bus

# **Algoritmo de Otimização Aritmética - Alocação de Filtro**

## Definição dos limites mínimos e máximos dos parâmetros

In [ ]:
R_shunt = list()
L_shunt = list()
C_shunt = list()

bases = [69,13.8,69,0.48,13.8,13.8,0.48,4.16,13.8,0.48,13.8,0.48,2.4]

for k in Dados_barra.index:
  if (Dados_barra['Tipo'][k]=='PQ' and ((np.abs(Estado_barra['Pk (pu)'][k])>=1e-4 and np.abs(Estado_barra['Qk (pu)'][k])>=1e-4))):
    Zbase = (bases[k-1]**2)/10
    if Estado_barra['Qk (pu)'][k]>=0:
      Z_serie_pu = np.conjugate((Estado_barra['V (pu)'][k]**2)/(-Estado_barra['Pk (pu)'][k] + 1j*Estado_barra['Qk (pu)'][k]))
      Z_serie_ohm = Z_serie_pu*Zbase
      R_shunt.append(np.real(Z_serie_ohm))
      C_shunt.append(1/(np.abs(np.imag(Z_serie_ohm))*(2*np.pi*60)))
    else:
      Z_serie_pu = np.conjugate((Estado_barra['V (pu)'][k]**2)/(-Estado_barra['Pk (pu)'][k] - 1j*Estado_barra['Qk (pu)'][k]))
      Z_serie_ohm = Z_serie_pu*Zbase
      R_shunt.append(np.real(Z_serie_ohm))
      L_shunt.append((np.imag(Z_serie_ohm)/(2*np.pi*60)))
  if (Dados_barra['Tipo'][k]=='PQ' and (np.abs(Dados_barra['Shunt (pu)'][k]!=0))):
    C_shunt.append(1/(((bases[k-1]**2)/(10*np.imag(Dados_barra['Shunt (pu)'][k])))*2*np.pi*60))

In [ ]:
Lmin = min(L_shunt)
Lmax = max(L_shunt)

Cmin = max(C_shunt)/10000000
Cmax = max(C_shunt)

Rmin = min(R_shunt)
Rmax = max(R_shunt)

## Dados de entrada da meta-heurística

- Vetor solução: [número_barra, R, L, C]
- Função objetivo:
 - **Caso 1:** Min $\displaystyle\sum_{k=1}^{N_{bus}} THD_{v}(k)$;
 - **Caso 2:** Min $\displaystyle\sum_{k=1}^{N_{bus}} \lambda(k)\cdot THD_{v}(k)$;
- Restrições:
 - $R_{min} \leq R \leq R_{max}$
 -  $L_{min} \leq L \leq L_{max}$
 - $C_{min} \leq C \leq C_{max}$
 - $\dot{V^{h}} = (Y_{barra}^{h})^{-1}\cdot [I_{inj}^{h} - I_{dem}^{h}]$

### Resultado do fluxo harmônico sem o filtro

In [ ]:
Res_h = fluxo_harmonico(Ynodal_h,Yshunt_h,Espectro_fonte,ordens_harmonicas,bus_harmonica,Dados_barra,Estado_barra)

In [ ]:
print('---------------- THD da tensão por barra ----------------')
for k in Dados_barra.index:
  print(f'Barra {k} - THDv (%) = {Res_h[1][str(k)]*100}')

### Metodologia para penalização da FOB

In [ ]:
# Definição da FOB

# Caso 1) Minimização do somatório de THD de tensão de todas as barras
# Caso 2) Minimização do somatório de THD de tensão de todas as barras considerando penalização

caso = 1

#--------------------------------------------------------------------------
if caso==1:

  # TODAS OS THDs de tensão são penalizados com valor unitário

  THD2 = [Res_h[1][str(bus)] for bus in Dados_barra.index] # todas as barra
  sum(THD2) # somatório dos THDs de todas as barras

#--------------------------------------------------------------------------

elif caso==2:

# Minimização do THD de tensão com diferentes penalizações desde o maior até
# o menor. Quanto maior o THD, maior a sua penalização.

  thd_barras = list()

  for k in range(1,np.size(Dados_barra,0)+1):
    thd_barras.append(Res_h[1][str(k)])

  somatorio_thd = sum(thd_barras)

  penalizacao = list()

  enderecos_thd_barras = list()

  # Coloca as barras em ordem decrescente de THD

  for k in range(0,np.size(Dados_barra,0)):
    enderecos_thd_barras.append(np.argmax(thd_barras)+1)
    thd_barras[np.argmax(thd_barras)] = -100000

  # Cria o vetor penalização de acordo com a % do somatório de THD

  fator = np.linspace(100,0,13)

  contador = 0
  for k in enderecos_thd_barras:
    penalizacao.append((Res_h[1][str(k)]/somatorio_thd)*fator[contador])
    contador = contador + 1

  # Função objetivo

  SUM_THD_PENALIZADO = 0
  contador = 0

  for k in enderecos_thd_barras:
    SUM_THD_PENALIZADO = SUM_THD_PENALIZADO + penalizacao[contador]*Res_h[1][str(k)]
    contador = contador + 1

### Chute inicial

In [ ]:
if caso==1:
  FOB_inicial = sum(THD2)
elif caso==2:
  FOB_inicial = SUM_THD_PENALIZADO

mu = 1.0 # parâmetro de ajuste no processo de busca

alpha = 16.5

epsilon = 1e-4 # constante para evitar divisão por zero

x_min = [[3,Rmin,Lmin,Cmin]] # valor mínimo das variáveis
x_max = [[13,Rmax,Lmax,Cmax]] # valor máximo das variáveis

N_solucoes = 100 # número de linhas da matriz solução
N_variaveis = np.size(x_min,1) # número de colunas = número de variáveis a serem calibradas

N_iter = 100 # Número total de iterações
iter = 0 # contador de iteracoes

x_inicial = np.zeros((N_solucoes,N_variaveis))

for i in range(0,np.size(x_inicial,0)):
  for j in range(0,np.size(x_inicial,1)):
    if j==0:
      x_inicial[i][j] = np.random.randint(x_min[0][j],x_max[0][j]+1) # chute inicial da alocação sorteio nas barras que possuem fonte harmônica
    else:
      x_inicial[i][j] = np.random.uniform(x_min[0][j],x_max[0][j])

x_sol = x_inicial.copy()
x_sol_new = np.zeros((N_solucoes,N_variaveis))

FOB = np.zeros((np.size(x_sol,0),1))
FOB_new = np.zeros((np.size(x_sol,0),1))

#-------------------- Inicialização da função objetivo -------------------------

for i in range(0,np.size(x_sol,0)):
  bus_filtro = x_sol[i][0] # Mudei aqui !!!!
  Zbase = (bases[int(bus_filtro-1)]**2)/10
  Yshunt_h_filtro = dict()
  for h in ordens_harmonicas:
    Yshunt_h_filtro[str(h)] = np.zeros((np.size(Yshunt_h[str(h)],0),np.size(Yshunt_h[str(h)],0)),dtype='complex')
    Yshunt_h_filtro[str(h)][int(bus_filtro-1)][int(bus_filtro-1)] = 1/((x_sol[i][1] + (1j*2*np.pi*60*h*x_sol[i][2]) + (1/(1j*2*np.pi*60*h*x_sol[i][3])))/Zbase)
    Yshunt_h_filtro[str(h)] = np.add(Yshunt_h[str(h)],Yshunt_h_filtro[str(h)])

  Fluxo_filtro = fluxo_harmonico(Ynodal_h,Yshunt_h_filtro,Espectro_fonte,ordens_harmonicas,bus_harmonica,Dados_barra,Estado_barra)

  if caso==1:

      #-------------------------- Somatório de THD ---------------------------------------------
      THD2_list = [Fluxo_filtro[1][str(barra)] for barra in Dados_barra.index] # todas as barras
      FOB[i][0] = sum(THD2_list) # todas as barra

  elif caso==2:

    #-------------------------- Penalização ---------------------------------------------

      SUM_THD_PENALIZADO = 0
      contador = 0
      for k in enderecos_thd_barras:
        SUM_THD_PENALIZADO = SUM_THD_PENALIZADO + penalizacao[contador]*Fluxo_filtro[1][str(k)]
        contador = contador + 1

      FOB[i][0] = SUM_THD_PENALIZADO

  #----------------------------------------------------------------------------------------

  if i==0:
    Best = FOB[i][0]
    Res_fluxo_harmonico_best = Fluxo_filtro
    linha = i
  elif FOB[i][0]<Best:
    Best = FOB[i][0]
    Res_fluxo_harmonico_best = Fluxo_filtro
    linha = i

#----------------- Vetores para visualização da convergência -------------------

x_label = np.zeros((1,N_iter+1))
y_label = np.zeros((1,N_iter+1))

#----------------- Inicialização dos vetores para plot -------------------------

x_label[0][iter] = iter
y_label[0][iter] = Best

In [ ]:
x_sol[linha] # print da melhor solução inicial

## Processo iterativo

In [ ]:
while iter<N_iter:

  iter = iter + 1

  AOM = 0.1666667 + (iter*(1 - 0.1666667 )/N_iter) # Acelerador de Otimização Matemático
  OMP = 1 - ((iter**(1/alpha))/(N_iter**(1/alpha))) # Otimizador Matemático de Probabilidade

  x_best = x_sol[linha]

  for i in range(0,np.size(x_sol,0)):

    r1 = np.random.rand(1,np.size(x_sol,1)) # valor randômico do processo
    r2 = np.random.rand(1,np.size(x_sol,1)) # valor randômico do processo
    r3 = np.random.rand(1,np.size(x_sol,1)) # valor randômico do processo

    for j in range(0,np.size(x_sol,1)):
      #------------------------------ ESPAÇO DE BUSCA GLOBAL ---------------------------
      if r1[0][j]>AOM:
        if r2[0][j]>0.5:
          if j==0:
            x_sol_new[i][j] = np.round(x_best[j]*OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j]))
          else:
            x_sol_new[i][j] = x_best[j]*OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j])
        else:
          if j==0:
            x_sol_new[i][j] = np.round(x_best[j]/OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j]))
          else:
            x_sol_new[i][j] = x_best[j]/OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j])
      #------------------------------ ESPAÇO DE BUSCA LOCAL ---------------------------
      else:
        if r3[0][j]>0.5:
          if j==0:
            x_sol_new[i][j] = np.round(x_best[j]+OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j]))
          else:
            x_sol_new[i][j] = x_best[j]+OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j])
        else:
          if j==0:
            x_sol_new[i][j] = np.round(x_best[j]-OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j]))
          else:
            x_sol_new[i][j] = x_best[j]-OMP*((x_max[0][j] - x_min[0][j])*mu + x_min[0][j])

      #------------------------------ Penalizações ---------------------------

      if x_sol_new[i][j]>x_max[0][j]:
        x_sol_new[i][j] = x_best[j]
      elif x_sol_new[i][j]<x_min[0][j]:
        x_sol_new[i][j] = x_best[j]

    #-------------------------------- Cálculo da FOB para cada solução atualizada --------------------------

    bus_filtro = x_sol_new[i][0]
    Zbase = (bases[int(bus_filtro-1)]**2)/10
    Yshunt_h_filtro = dict()
    for h in ordens_harmonicas:
      Yshunt_h_filtro[str(h)] = np.zeros((np.size(Yshunt_h[str(h)],0),np.size(Yshunt_h[str(h)],0)),dtype='complex')
      Yshunt_h_filtro[str(h)][int(bus_filtro-1)][int(bus_filtro-1)] = 1/((x_sol_new[i][1] + (1j*2*np.pi*60*h*x_sol_new[i][2]) + (1/(1j*2*np.pi*60*h*x_sol_new[i][3])))/Zbase)
      Yshunt_h_filtro[str(h)] = np.add(Yshunt_h[str(h)],Yshunt_h_filtro[str(h)])

    Fluxo_filtro = fluxo_harmonico(Ynodal_h,Yshunt_h_filtro,Espectro_fonte,ordens_harmonicas,bus_harmonica,Dados_barra,Estado_barra)

    if caso==1:

        #-------------------------- Somatório de THD ---------------------------------------------
        THD2_list = [Fluxo_filtro[1][str(barra)] for barra in Dados_barra.index] # todas as barras
        FOB_new[i][0] = sum(THD2_list) # todas as barra

    elif caso==2:

      #-------------------------- Penalização ---------------------------------------------

        SUM_THD_PENALIZADO = 0
        contador = 0
        for k in enderecos_thd_barras:
          SUM_THD_PENALIZADO = SUM_THD_PENALIZADO + penalizacao[contador]*Fluxo_filtro[1][str(k)]
          contador = contador + 1

        FOB_new[i][0] = SUM_THD_PENALIZADO

    #----------------------------------------------------------------------------------------

    if FOB_new[i][0]<FOB[i][0]:
      x_sol[i] = x_sol_new[i]
      FOB[i][0] = FOB_new[i][0]

  #-------------------------------- Determinação da melhor solução na iteração atual ----------------------

  for s in range(0,np.size(FOB,0)):
    if FOB[s][0]<Best:
      Best = FOB[s][0]
      linha = s

  x_label[0][iter] = iter
  y_label[0][iter] = Best

  print(f'Best(iter = {iter}) = {Best}')

## Resultados - AOA

In [ ]:
Dimensionamento_otimo = pd.DataFrame(data=np.transpose(x_best),index=['Barra','R (Ohm)','L (Henry)','C (Farad)'],columns=['Valores ótimos'])
Dimensionamento_otimo

In [ ]:
f_sintonia = (1/np.sqrt(x_best[2]*x_best[3]))/(2*np.pi)/60

print(f'Ordem harmônica sintonizada = {(f_sintonia)}')

In [ ]:
# -------------------> Cálculo do fluxo harmônico com o filtro dimensionado e alocado

Yshunt_h_filtro = dict()
Zbase = (bases[int(x_best[0]-1)]**2)/10
for h in ordens_harmonicas:
  Yshunt_h_filtro[str(h)] = np.zeros((np.size(Yshunt_h[str(h)],0),np.size(Yshunt_h[str(h)],0)),dtype='complex')
  Yshunt_h_filtro[str(h)][int(x_best[0]-1)][int(x_best[0]-1)] = 1/((x_best[1] + (1j*2*np.pi*60*h*x_best[2]) + (1/(1j*2*np.pi*60*h*x_best[3])))/Zbase)
  Yshunt_h_filtro[str(h)] = np.add(Yshunt_h[str(h)],Yshunt_h_filtro[str(h)])

fluxo_h_filtro = fluxo_harmonico(Ynodal_h,Yshunt_h_filtro,Espectro_fonte,ordens_harmonicas,bus_harmonica,Dados_barra,Estado_barra)

In [ ]:
#--------------------- Cálculo das perdas antes da alocação do filtro ------------------------

Sk_antes = 0

for k in ordens_harmonicas:
  Sk_antes = Sk_antes + np.dot(np.transpose(Res_h[0][str(k)]),np.conjugate(Res_h[3][str(k)]))[0][0]

#--------------------- Cálculo das perdas após alocação do filtro ------------------------

Sk_pos = 0

for k in ordens_harmonicas:
  Sk_pos = Sk_pos + np.dot(np.transpose(fluxo_h_filtro[0][str(k)]),np.conjugate(fluxo_h_filtro[3][str(k)]))[0][0]

In [ ]:
print(f'Perdas elétricas antes = {np.real(Sk_antes)} pu')
print(f'Perdas elétricas depois = {np.real(Sk_pos)} pu')

# **Gráficos**

## Convergência do AOA

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))

x = list()
y = list()

for k in range(0,np.size(x_label,1)):
  x.append(x_label[0][k])
  y.append(y_label[0][k])

ax.plot(x,y,'ro-',lw=1.25,label='Processo de convergência');

plt.xticks(np.linspace(0,N_iter,11));
plt.yticks(np.linspace(min(y),y[0],15));
ax.set_ylabel('Função objetivo - Min THD');
ax.set_xlabel('Iteração');
ax.set_title('Convergência do Algoritmo de Otimização Aritmética')
ax.legend();

## THD por barra antes e após a correção

In [ ]:
X = list()

for k in range(1,np.size(Dados_barra,0)+1):
  X.append(int(k))

THD_antes = list()
THD_depois = list()

for k in Dados_barra.index:
  THD_antes.append(Res_h[1][str(k)]*100)
  THD_depois.append(fluxo_h_filtro[1][str(k)]*100)

X_axis = np.arange(len(X))

plt.subplots(figsize=(6,4))

plt.bar(X_axis - 0.15, THD_antes, 0.3, label = 'Sem filtro',color='blue',edgecolor='black')
plt.bar(X_axis + 0.15, THD_depois, 0.3, label = 'Com filtro',color='lightseagreen',edgecolor='black')

max_antes = max(THD_antes)
max_depois = max(THD_depois)

plt.xticks(X_axis, X)
plt.xlabel("Barra")
plt.ylabel("THD (%)")
plt.ylim([0,max(max_antes,max_depois) + 3])
plt.yticks(np.linspace(0,max(THD_antes) + 3,15))
plt.title('THD (%) por barra')
plt.legend()
plt.show()

In [ ]:
print('--------------- THDv das barras após alocação do filtro ---------------')
for k in range(1,np.size(Dados_barra,0)+1):
  print(f'Barra {k} - THDv (%) = {Fluxo_filtro[1][str(k)]*100}')

## Comparação THD por barra e modelagem da Função Objetivo

- **Resultado dos THD's de tensão e perdas para cada tipo de modelagem da FOB**

In [ ]:
if caso==1:
  THD_caso_1 = list()
  for k in Dados_barra.index:
    THD_caso_1.append(fluxo_h_filtro[1][str(k)]*100)
  Perdas_caso_1 = np.real(Sk_pos)
elif caso==2:
  THD_caso_2 = list()
  for k in Dados_barra.index:
    THD_caso_2.append(fluxo_h_filtro[1][str(k)]*100)
  Perdas_caso_2 = np.real(Sk_pos)

 - **Gráficos que representam as distorções harmônicas totais de tensão após a alocação do filtro para cada tipo de caso estudado**

In [ ]:
X_axis = np.arange(len(X))

plt.subplots(figsize=(6,4))

plt.bar(X_axis - 0.20, THD_caso_1, 0.40, label = 'Caso 1',color='blue',edgecolor='black')
plt.bar(X_axis + 0.20, THD_caso_2, 0.40, label = 'Caso 2',color='red',edgecolor='black')

max_caso_1 = max(THD_caso_1)
max_caso_2 = max(THD_caso_2)

plt.xticks(X_axis, X)
plt.xlabel("Barra")
plt.ylabel("THD (%)")
plt.ylim([0,max(max_caso_1,max_caso_2) + 3])
plt.yticks(np.linspace(0,max(max_caso_1,max_caso_2) + 3,15))
plt.title('THD (%) por barra após alocação do filtro')
plt.legend()
plt.show()

- **Gráficos que representam as perdas elétricas após a alocação do filtro para cada caso estudado**

In [ ]:
X_axis = np.arange(len([1]))

plt.subplots(figsize=(6,4))

plt.bar(X_axis - 0.1, Perdas_caso_1*10000, 0.2, label = 'Caso 1',color='blue',edgecolor='black')
plt.bar(X_axis + 0.1, Perdas_caso_2*10000, 0.2, label = 'Caso 2',color='red',edgecolor='black')

max_caso_1 = Perdas_caso_1*10000
max_caso_2 = Perdas_caso_2*10000

plt.xticks(X_axis, [' '])
plt.xlabel(" ")
plt.ylabel("Perdas elétricas (kW)")
plt.ylim([0,max(max_caso_1,max_caso_2) + 10])
plt.yticks(np.linspace(0,max(max_caso_1,max_caso_2) + 10,10))
plt.title('Perdas elétricas após a alocação do filtro')
plt.legend()
plt.show()